In [12]:
import chess
import random
import chess.engine
import chess.svg

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import tensorflow as tf 
import pandas as pd
import numpy as np

def convertToFeatureVector(boardState):
    pieces = []
    encoding = {'P':1,'R':2,'N':3,'B':4,'Q':5,'K':6,'p':-1,'r':-2,'n':-3,'b':-4,'q':-5,'k':-6, '.':0}
    for i in range(64):
        piece = boardState.piece_at(i)
        if piece:
            piece = piece.symbol()
        else:
            piece = '.'
        pieces.append(encoding[piece])
    return pieces

print("Loading Model")
model = tf.keras.models.load_model("deepchess2.h5")
print("Done Loading")

def evaluateMove(move1, move2):
    #simple_dense.h5
    
    df1 = pd.DataFrame(data=[move1])
    df2 = pd.DataFrame(data=[move2])
    return model.predict([df1, df2])

def countMoves(moves):
    length = 0
    for _ in moves:
        length += 1
    return length

winCount = 0
printStr = ""
for _ in range(0, 1):

    board = chess.Board()
    engine = chess.engine.SimpleEngine.popen_uci("/usr/local/bin/stockfish")
    for i in range(0, 4):
        moves = board.legal_moves
        
        moveCount = countMoves(moves)
        randomIndex = random.randrange(0,moveCount,1)

        count = 0
        for move in moves:
            if(count == randomIndex):
                board.push(move)
                break
            count += 1

    '''
    nnWhite = True
    if (random.random() < .5):
        result = engine.play(board, chess.engine.Limit(time=0.1))
        board.push(result.move)
        nnWhite = False
    '''

    while not board.is_game_over():
        
        ogBoard = board
        count = 0
        move1 = move2 = None
        board1 = board2 = None

        for move in ogBoard.legal_moves:
            if count == 0:
                move1 = move
            else:
                modBoard = ogBoard
                move2 = move
                
                modBoard.push(move1)
                move1Feature = convertToFeatureVector(modBoard)
                modBoard.pop()
                
                modBoard.push(move2)
                move2Feature = convertToFeatureVector(modBoard)
                modBoard.pop()

                strengthArr = evaluateMove(move1Feature, move2Feature)[0]

                if(strengthArr[0] < strengthArr[1]):
                    move1 = move2

                #compare
            count += 1

        board.push(move1)

        if(board.is_game_over()):
            break;
        
        result = engine.play(board, chess.engine.Limit(depth=1))
        board.push(result.move)
        
    display(chess.svg.board(board=board))
    printStr = printStr+""+board.result()+"\n"
    engine.quit()

print(printStr)


Loading Model


OSError: Unable to open file (unable to open file: name = 'deepchess2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)